In [1]:
# Import dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from config import google_api_key
import json
import requests
import sqlite3
from sqlalchemy import create_engine

## 1. Extract city and state based on zip code

### 1.1. Loading raw data

In [2]:
# Load dataset
df = pd.read_csv("../data/lender_data.csv", skiprows=1, dtype={'Zip': object})

# Drop rows that are all NaN's
df = df.dropna(how='all')
df.head()

,Deal ID,Lender ID,Zip,City,State,Multifamily Subtype,Built,Units,Original Loan,Note Rate,Loan Term (Original),Appraised Value,Maturity Date,UPB,Amort DSvc,Orig Amort,IO Period,Unnamed: 17,Unnamed: 18
0,14454.0,19.0,92626,NaN,NaN,Garden,1990,24.0,"1,720,000",5.8%,180.0,"3,500,000",2/1/19,"28,379","14,292",180.0,0.0,NaN,NaN
1,20421.0,19.0,91406,NaN,NaN,Garden,1962,15.0,"500,000",5.3%,180.0,"1,525,000",12/1/19,"46,916","4,022",180.0,0.0,NaN,NaN
2,24298.0,50.0,70122,NaN,NaN,Garden,2009,164.0,"250,000",6.9%,120.0,"4,187,586",1/1/21,"67,161","2,892",120.0,0.0,NaN,NaN
3,2853.0,19.0,90046,NaN,NaN,Garden,1957,11.0,"805,000",5.2%,180.0,"2,450,000",8/1/20,"123,314","6,450",180.0,0.0,NaN,NaN
4,3023.0,51.0,11215,NaN,NaN,Garden,2002,13.0,"1,600,000",6.5%,120.0,"3,112,000",7/1/19,"124,669","18,200",120.0,0.0,NaN,NaN


In [3]:
# Drop meaningless columns "Unnamed: 17", "Unnamed: 18"
df = df.drop(columns=[col for col in df.columns.tolist() if 'Unnamed' in col])

# Overview of dataset
print(f"There are {df.shape[0]} rows in the dataset.")

# Number of missing values in each column
print("Number of missing values in each column:")
df.isnull().sum()

There are 29130 rows in the dataset.
Number of missing values in each column:


Deal ID                     0
Lender ID                   0
Zip                         0
City                    29130
State                   29130
Multifamily Subtype         0
Built                       0
 Units                      0
Original Loan               0
Note Rate                   0
Loan Term (Original)        0
Appraised Value             0
Maturity Date               0
UPB                         0
Amort DSvc                  0
Orig Amort                  0
IO Period                   0
dtype: int64

In [4]:
# Number of unique values
print("Number of unique values in each column:")
df.nunique()

Number of unique values in each column:


Deal ID                 29128
Lender ID                  58
Zip                      6974
City                        0
State                       0
Multifamily Subtype        20
Built                     165
 Units                    886
Original Loan           12396
Note Rate                  77
Loan Term (Original)       90
Appraised Value          7022
Maturity Date             387
UPB                     25261
Amort DSvc              25552
Orig Amort                 45
IO Period                  35
dtype: int64

### 1.2. Fill in city and state blanks

Use [Google Geocoding API](https://developers.google.com/maps/documentation/geocoding/intro) to get the City and State for each deal from the Zip Code provided.

In [5]:
# Construct API query
base_url = "https://maps.googleapis.com/maps/api/geocode/json"
params = {"key": google_api_key}

In [6]:
# Create a list of unique zip code to look up
zipcode_unique = df['Zip'].unique().tolist()

In [7]:
# results_list = []
# for index, zipcode in enumerate(zipcode_unique):
    
#     target_zipcode = zipcode
#     params['components'] = f"country:US|postal_code:{target_zipcode}"
#     print(f"Retrieving Results for Index {index}: {target_zipcode}.")

#     response = requests.get(base_url, params=params).json()
#     results = response.get('results')
#     results_list.append(results)

Retrieving Results for Index 0: 92626.
Retrieving Results for Index 1: 91406.
Retrieving Results for Index 2: 70122.
Retrieving Results for Index 3: 90046.
Retrieving Results for Index 4: 11215.
Retrieving Results for Index 5: 66062.
Retrieving Results for Index 6: 90245.
Retrieving Results for Index 7: 13601.
Retrieving Results for Index 8: 95117.
Retrieving Results for Index 9: 31806.
Retrieving Results for Index 10: 90068.
Retrieving Results for Index 11: 90262.
Retrieving Results for Index 12: 31096.
Retrieving Results for Index 13: 31002.
Retrieving Results for Index 14: 91204.
Retrieving Results for Index 15: 30428.
Retrieving Results for Index 16: 30411.
Retrieving Results for Index 17: 31824.
Retrieving Results for Index 18: 02062.
Retrieving Results for Index 19: 31091.
Retrieving Results for Index 20: 30810.
Retrieving Results for Index 21: 83631.
Retrieving Results for Index 22: 98208.
Retrieving Results for Index 23: 83318.
Retrieving Results for Index 24: 31650.
Retrieving

Retrieving Results for Index 203: 97383.
Retrieving Results for Index 204: 90301.
Retrieving Results for Index 205: 45219.
Retrieving Results for Index 206: 55428.
Retrieving Results for Index 207: 72160.
Retrieving Results for Index 208: 28083.
Retrieving Results for Index 209: 90027.
Retrieving Results for Index 210: 90621.
Retrieving Results for Index 211: 78208.
Retrieving Results for Index 212: 93033.
Retrieving Results for Index 213: 92109.
Retrieving Results for Index 214: 70805.
Retrieving Results for Index 215: 91706.
Retrieving Results for Index 216: 98662.
Retrieving Results for Index 217: 90815.
Retrieving Results for Index 218: 71115.
Retrieving Results for Index 219: 61820.
Retrieving Results for Index 220: 07050.
Retrieving Results for Index 221: 92025.
Retrieving Results for Index 222: 90804.
Retrieving Results for Index 223: 60415.
Retrieving Results for Index 224: 60506.
Retrieving Results for Index 225: 92105.
Retrieving Results for Index 226: 97502.
Retrieving Resul

Retrieving Results for Index 403: 92110.
Retrieving Results for Index 404: 90250.
Retrieving Results for Index 405: 63110.
Retrieving Results for Index 406: 36567.
Retrieving Results for Index 407: 01040.
Retrieving Results for Index 408: 07042.
Retrieving Results for Index 409: 94610.
Retrieving Results for Index 410: 39846.
Retrieving Results for Index 411: 02904.
Retrieving Results for Index 412: 44112.
Retrieving Results for Index 413: 92345.
Retrieving Results for Index 414: 97322.
Retrieving Results for Index 415: 83854.
Retrieving Results for Index 416: 55435.
Retrieving Results for Index 417: 74074.
Retrieving Results for Index 418: 83706.
Retrieving Results for Index 419: 60659.
Retrieving Results for Index 420: 11238.
Retrieving Results for Index 421: 78578.
Retrieving Results for Index 422: 27405.
Retrieving Results for Index 423: 11216.
Retrieving Results for Index 424: 16001.
Retrieving Results for Index 425: 08609.
Retrieving Results for Index 426: 44109.
Retrieving Resul

Retrieving Results for Index 603: 96815.
Retrieving Results for Index 604: 30533.
Retrieving Results for Index 605: 29527.
Retrieving Results for Index 606: 32304.
Retrieving Results for Index 607: 71602.
Retrieving Results for Index 608: 84041.
Retrieving Results for Index 609: 97232.
Retrieving Results for Index 610: 28625.
Retrieving Results for Index 611: 85014.
Retrieving Results for Index 612: 98363.
Retrieving Results for Index 613: 98407.
Retrieving Results for Index 614: 21230.
Retrieving Results for Index 615: 85719.
Retrieving Results for Index 616: 48174.
Retrieving Results for Index 617: 01833.
Retrieving Results for Index 618: 80220.
Retrieving Results for Index 619: 98368.
Retrieving Results for Index 620: 35243.
Retrieving Results for Index 621: 07501.
Retrieving Results for Index 622: 99501.
Retrieving Results for Index 623: 33135.
Retrieving Results for Index 624: 94116.
Retrieving Results for Index 625: 84102.
Retrieving Results for Index 626: 14607.
Retrieving Resul

Retrieving Results for Index 804: 32536.
Retrieving Results for Index 805: 97401.
Retrieving Results for Index 806: 32901.
Retrieving Results for Index 807: 37160.
Retrieving Results for Index 808: 32803.
Retrieving Results for Index 809: 97702.
Retrieving Results for Index 810: 02134.
Retrieving Results for Index 811: 54409.
Retrieving Results for Index 812: 47150.
Retrieving Results for Index 813: 94121.
Retrieving Results for Index 814: 92655.
Retrieving Results for Index 815: 27707.
Retrieving Results for Index 816: 32608.
Retrieving Results for Index 817: 91803.
Retrieving Results for Index 818: 57032.
Retrieving Results for Index 819: 97701.
Retrieving Results for Index 820: 07505.
Retrieving Results for Index 821: 29670.
Retrieving Results for Index 822: 70127.
Retrieving Results for Index 823: 11218.
Retrieving Results for Index 824: 04083.
Retrieving Results for Index 825: 90018.
Retrieving Results for Index 826: 91103.
Retrieving Results for Index 827: 93402.
Retrieving Resul

...

Retrieving Results for Index 6107: 46122.
Retrieving Results for Index 6108: 85748.
Retrieving Results for Index 6109: 18508.
Retrieving Results for Index 6110: 02301.
Retrieving Results for Index 6111: 63021.
Retrieving Results for Index 6112: 48075.
Retrieving Results for Index 6113: 32730.
Retrieving Results for Index 6114: 60504.
Retrieving Results for Index 6115: 97351.
Retrieving Results for Index 6116: 32614.
Retrieving Results for Index 6117: 23114.
Retrieving Results for Index 6118: 23693.
Retrieving Results for Index 6119: 70817.
Retrieving Results for Index 6120: 34714.
Retrieving Results for Index 6121: 64158.
Retrieving Results for Index 6122: 83716.
Retrieving Results for Index 6123: 43209.
Retrieving Results for Index 6124: 35214.
Retrieving Results for Index 6125: 75024.
Retrieving Results for Index 6126: 33411.
Retrieving Results for Index 6127: 63124.
Retrieving Results for Index 6128: 85382.
Retrieving Results for Index 6129: 94015.
Retrieving Results for Index 6130:

Retrieving Results for Index 6303: 80465.
Retrieving Results for Index 6304: 94555.
Retrieving Results for Index 6305: 80026.
Retrieving Results for Index 6306: 15222.
Retrieving Results for Index 6307: 68510.
Retrieving Results for Index 6308: 40243.
Retrieving Results for Index 6309: 73108.
Retrieving Results for Index 6310: 92065.
Retrieving Results for Index 6311: 30338.
Retrieving Results for Index 6312: 59405.
Retrieving Results for Index 6313: 32311.
Retrieving Results for Index 6314: 32765.
Retrieving Results for Index 6315: 06810.
Retrieving Results for Index 6316: 49002.
Retrieving Results for Index 6317: 96753.
Retrieving Results for Index 6318: 01375.
Retrieving Results for Index 6319: 38680.
Retrieving Results for Index 6320: 77498.
Retrieving Results for Index 6321: 49428.
Retrieving Results for Index 6322: 98597.
Retrieving Results for Index 6323: 53142.
Retrieving Results for Index 6324: 95377.
Retrieving Results for Index 6325: 22408.
Retrieving Results for Index 6326:

Retrieving Results for Index 6499: 44502.
Retrieving Results for Index 6500: 46037.
Retrieving Results for Index 6501: 80921.
Retrieving Results for Index 6502: 37219.
Retrieving Results for Index 6503: 60040.
Retrieving Results for Index 6504: 32606.
Retrieving Results for Index 6505: 18966.
Retrieving Results for Index 6506: 20772.
Retrieving Results for Index 6507: 01970.
Retrieving Results for Index 6508: 21128.
Retrieving Results for Index 6509: 21108.
Retrieving Results for Index 6510: 29935.
Retrieving Results for Index 6511: 35223.
Retrieving Results for Index 6512: 92673.
Retrieving Results for Index 6513: 33484.
Retrieving Results for Index 6514: 02466.
Retrieving Results for Index 6515: 08205.
Retrieving Results for Index 6516: 51106.
Retrieving Results for Index 6517: 75048.
Retrieving Results for Index 6518: 91913.
Retrieving Results for Index 6519: 01060.
Retrieving Results for Index 6520: 77565.
Retrieving Results for Index 6521: 02840.
Retrieving Results for Index 6522:

Retrieving Results for Index 6695: 20120.
Retrieving Results for Index 6696: 76182.
Retrieving Results for Index 6697: 92590.
Retrieving Results for Index 6698: 98271.
Retrieving Results for Index 6699: 49321.
Retrieving Results for Index 6700: 08857.
Retrieving Results for Index 6701: 33705.
Retrieving Results for Index 6702: 20874.
Retrieving Results for Index 6703: 77005.
Retrieving Results for Index 6704: 30305.
Retrieving Results for Index 6705: 23510.
Retrieving Results for Index 6706: 11210.
Retrieving Results for Index 6707: 32250.
Retrieving Results for Index 6708: 92604.
Retrieving Results for Index 6709: 80923.
Retrieving Results for Index 6710: 01923.
Retrieving Results for Index 6711: 78738.
Retrieving Results for Index 6712: 89149.
Retrieving Results for Index 6713: 48105.
Retrieving Results for Index 6714: 22079.
Retrieving Results for Index 6715: 33027.
Retrieving Results for Index 6716: 76087.
Retrieving Results for Index 6717: 06042.
Retrieving Results for Index 6718:

Retrieving Results for Index 6891: 75028.
Retrieving Results for Index 6892: 92130.
Retrieving Results for Index 6893: 01605.
Retrieving Results for Index 6894: 95757.
Retrieving Results for Index 6895: 33435.
Retrieving Results for Index 6896: 95742.
Retrieving Results for Index 6897: 19129.
Retrieving Results for Index 6898: 94582.
Retrieving Results for Index 6899: 80924.
Retrieving Results for Index 6900: 92707.
Retrieving Results for Index 6901: 07470.
Retrieving Results for Index 6902: 33445.
Retrieving Results for Index 6903: 08096.
Retrieving Results for Index 6904: 44124.
Retrieving Results for Index 6905: 20817.
Retrieving Results for Index 6906: 01864.
Retrieving Results for Index 6907: 33018.
Retrieving Results for Index 6908: 34212.
Retrieving Results for Index 6909: 34119.
Retrieving Results for Index 6910: 22302.
Retrieving Results for Index 6911: 10038.
Retrieving Results for Index 6912: 91739.
Retrieving Results for Index 6913: 92612.
Retrieving Results for Index 6914:

In [8]:
# # Save results from API requests
# with open('zipcode_locations.json', 'w') as file_out:
#     json.dump(results_list, file_out)

In [7]:
# Load results from API requests
with open('temp/zipcode_locations.json', 'r') as file_in:
    zipcode_locations = json.load(file_in)

In [8]:
# Example API results
zipcode_locations[0]

[{'address_components': [{'long_name': '92626',
    'short_name': '92626',
    'types': ['postal_code']},
   {'long_name': 'Costa Mesa',
    'short_name': 'Costa Mesa',
    'types': ['locality', 'political']},
   {'long_name': 'Orange County',
    'short_name': 'Orange County',
    'types': ['administrative_area_level_2', 'political']},
   {'long_name': 'California',
    'short_name': 'CA',
    'types': ['administrative_area_level_1', 'political']},
   {'long_name': 'United States',
    'short_name': 'US',
    'types': ['country', 'political']}],
  'formatted_address': 'Costa Mesa, CA 92626, USA',
  'geometry': {'bounds': {'northeast': {'lat': 33.701975, 'lng': -117.8642701},
    'southwest': {'lat': 33.65618, 'lng': -117.946433}},
   'location': {'lat': 33.6834142, 'lng': -117.9073244},
   'location_type': 'APPROXIMATE',
   'viewport': {'northeast': {'lat': 33.701975, 'lng': -117.8642701},
    'southwest': {'lat': 33.65618, 'lng': -117.946433}}},
  'place_id': 'ChIJj4r82UHf3IARLQc40-4

According to the Google API documentation, `locality` is the address type that corresponds to city, and `administrative_area_level_1` is the address type that corresponds to state. However, sometimes `locality` does not exist in API results. Ideally, I would look up their exact city names. For the sake of time in this project, I'm going to use the second element in the `address_components` list as a proxy for the city. 

In [9]:
# Define a function to retrieve city and state information from API results of a particular zip code
def retrieve_city_state(index, results, zipcode_unique=zipcode_unique):
    '''
    Retrieve city and state information from API results
    INPUTS:
        index (int) - index of the particular zip code in `zipcode_unique`
        results (list) - list of API results of the particular zip code
        zipcode_unique (list) - list of unique zip code in the raw data
    OUTPUTS:
        city (str or NaN) - name of the corresponding city
        state (str or NaN) - name of the corresponding state
    '''
    city = np.nan
    state = np.nan
    if (results):
        address = results[0].get("address_components", "")
            
        for component in address:
            if 'locality' in component['types']:
                city = component.get("short_name", "") # protect from keyError
            elif 'administrative_area_level_1' in component['types']:
                state = component.get("short_name", "")
        
        # If locality does not exist in the API results
        if city is np.nan:
            city = address[1].get("short_name", "")
    else:
        print(f"No results for Index {index}: {zipcode_unique[index]}")
        
    return city, state

In [10]:
# Retrieve city and state information from API results
city_list = []
state_list = []

for index, results in enumerate(zipcode_locations):

    city, state = retrieve_city_state(index, results, zipcode_unique=zipcode_unique)
    city_list.append(city)
    state_list.append(state)

No results for Index 4623: 85220
No results for Index 5886: 85219
No results for Index 6020: 85223
No results for Index 6268: 85242


It appears that zip code 85220, 85219, 85223, and 85242 do not have valid city and state information. A closer look up on [USPS website](https://about.usps.com/postal-bulletin/2009/pb22259/html/info_001.htm) shows that these are the old codes and should be updated to the new ones: 85120, 85119, 85123, and 85142.

In [11]:
# Update zip codes and retrieve city and state info again
indices = [4623, 5886, 6020, 6268]
old_zipcode = [85220, 85219, 85223, 85242]
new_zipcode = [85120, 85119, 85123, 85142]

for i, index in enumerate(indices):
    
    # API request using the updated zip codes
    target_zipcode = new_zipcode[i]
    params['components'] = f"country:US|postal_code:{target_zipcode}"
    print(f"Retrieving Results for Index {index}: {target_zipcode}.")
    response = requests.get(base_url, params=params).json()
    results = response.get('results')
    
    # Retrieve city and state information
    city, state = retrieve_city_state(index, results, zipcode_unique=zipcode_unique)
    city_list[index] = city
    state_list[index] = state

Retrieving Results for Index 4623: 85120.
Retrieving Results for Index 5886: 85119.
Retrieving Results for Index 6020: 85123.
Retrieving Results for Index 6268: 85142.


In [12]:
# Combine zipcode, city, and state info into dictionary for easy query
zipcode_dict = {}
for index, zipcode in enumerate(zipcode_unique):
    zipcode_dict[zipcode] = {'city': city_list[index], 'state': state_list[index]}

In [13]:
# Fill in city and state info based on zip code
df['City'] = df['Zip'].apply(lambda x: zipcode_dict[x]['city'])
df['State'] = df['Zip'].apply(lambda x: zipcode_dict[x]['state'])
df.head()

,Deal ID,Lender ID,Zip,City,State,Multifamily Subtype,Built,Units,Original Loan,Note Rate,Loan Term (Original),Appraised Value,Maturity Date,UPB,Amort DSvc,Orig Amort,IO Period
0,14454.0,19.0,92626,Costa Mesa,CA,Garden,1990,24.0,"1,720,000",5.8%,180.0,"3,500,000",2/1/19,"28,379","14,292",180.0,0.0
1,20421.0,19.0,91406,Los Angeles,CA,Garden,1962,15.0,"500,000",5.3%,180.0,"1,525,000",12/1/19,"46,916","4,022",180.0,0.0
2,24298.0,50.0,70122,New Orleans,LA,Garden,2009,164.0,"250,000",6.9%,120.0,"4,187,586",1/1/21,"67,161","2,892",120.0,0.0
3,2853.0,19.0,90046,West Hollywood,CA,Garden,1957,11.0,"805,000",5.2%,180.0,"2,450,000",8/1/20,"123,314","6,450",180.0,0.0
4,3023.0,51.0,11215,Brooklyn,NY,Garden,2002,13.0,"1,600,000",6.5%,120.0,"3,112,000",7/1/19,"124,669","18,200",120.0,0.0


In [14]:
# Sanity check
print(f"The dataset has {df['State'].nunique()} unique states.")

The dataset has 51 unique states.


### 1.3. Sanity check of the table

In [15]:
# Column name
df.columns

Index(['Deal ID', 'Lender ID', 'Zip', 'City', 'State', 'Multifamily Subtype',
       'Built', ' Units', 'Original Loan ', 'Note Rate',
       'Loan Term (Original)', 'Appraised Value', 'Maturity Date', 'UPB',
       'Amort DSvc', 'Orig Amort', 'IO Period'],
      dtype='object')

In [16]:
# Clean up column name
df.columns = [col.strip() for col in df.columns]
df.columns

Index(['Deal ID', 'Lender ID', 'Zip', 'City', 'State', 'Multifamily Subtype',
       'Built', 'Units', 'Original Loan', 'Note Rate', 'Loan Term (Original)',
       'Appraised Value', 'Maturity Date', 'UPB', 'Amort DSvc', 'Orig Amort',
       'IO Period'],
      dtype='object')

In [17]:
# Check the number of missing values in Lender ID
id_col = 'Lender ID'
missing_id = df.loc[(df[id_col].isnull()) | (df[id_col] == ""), id_col]
print(f"Number of missing values in Lender ID column: {missing_id.count()}")

Number of missing values in Lender ID column: 0


/anaconda3/envs/py36/lib/python3.6/site-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [18]:
# Check the number of missing values in other features
print("Number of missing values in each column:")
df.isnull().sum()

Number of missing values in each column:


Deal ID                 0
Lender ID               0
Zip                     0
City                    0
State                   0
Multifamily Subtype     0
Built                   0
Units                   0
Original Loan           0
Note Rate               0
Loan Term (Original)    0
Appraised Value         0
Maturity Date           0
UPB                     0
Amort DSvc              0
Orig Amort              0
IO Period               0
dtype: int64

In [19]:
# Drop duplicated rows
df = df.drop_duplicates()
print(f"Number of duplicated rows left: {df.duplicated().sum()}")

Number of duplicated rows left: 0


At this point, no missing value or duplicated row.

### 1.4. Save checkpoint

In [20]:
# Save dataframe into sqlite
engine = create_engine('sqlite:///../data/lender_data1.db')
df.to_sql('Lender', engine, index=False, if_exists='replace')
# engine.dispose()

# Save as a checkpoint
df.to_csv("temp/lender_checkpoint_1.csv", index=False)